# LlamaIndex / Predibase Integration

This notebook shows how you can use [Predibase](https://predibase.com) to manage the prompts you use within LlamaIndex. By registering LlamaIndex prompts with Vellum via the `VellumPredictor` class, you're able to use Vellum to:
1. View and manage all your prompts in a UI
2. Compare and constrast prompt templates, and even different LLM providers/models side-by-side
3. Update your prompt, or the provider/model backing it, without updating or re-deploying code
4. Set up unit tests to test your prompt and quantitatively evaluate the quality of its outputs
5. Observe and monitor all traffic to it, including the inputs sent to the model, and the outputs sent back

## Getting Started

1. Sign up for a free Predibase account [here](https://predibase.com/free-trial)
2. Create an Account
3. Go to Settings > My profile and Generate a new API Token.

In [ ]:
!pip install llama-index --quiet
!pip install predibase --quiet
!pip install sentence-transformers --quiet

In [ ]:
import os

os.environ[
    "PREDIBASE_API_TOKEN"
] = "{PREDIBASE_API_TOKEN}"
from llama_index.llms import PredibaseLLM

## Flow 1: Query Predibase LLM directly

In [ ]:
llm = PredibaseLLM(model_name="llama-2-13b", temperature=0.3, max_new_tokens=512)
# You can query any HuggingFace or fine-tuned LLM that's hosted on Predibase

In [ ]:
result = llm.complete("Can you recommend me a nice dry white wine?")
print(result)

## Flow 2: Retrieval Augmented Generation (RAG) with Predibase LLM

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.embeddings import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

### Load Documents

In [ ]:
documents = SimpleDirectoryReader("../paul_graham_essay/data").load_data()

### Configure Predibase LLM

In [ ]:
llm = PredibaseLLM(
    model_name="llama-2-13b", temperature=0.3, max_new_tokens=400, context_window=1024
)
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())
service_context = ServiceContext.from_defaults(
    chunk_size=1024, llm=llm, embed_model=embed_model
)

### Setup and Query Index

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")

In [ ]:
print(response)